In [12]:
from random import Random
from numpy import *
from time import *
from datetime import *
from BlackScholes import *
import math
import scipy as sp

import matplotlib.pyplot as plt
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
KC = 130
KP = 70
r = 0.002
sigma = 0.4
T = 2.0
S0 = 100

In [18]:
lb = 0
ub = 300
training_number = 30
testing_number = 100

call = lambda x,y: bsformula(1, lb+(ub-lb)*x, KC, r, T, y, 0)[0]
put = lambda x,y: bsformula(-1, lb+(ub-lb)*x, KP, r, T, y, 0)[0]

# delta

In [34]:
x_train = np.array(np.linspace(0.01,1.0, training_number), dtype='float32').reshape(training_number, 1)
x_test = np.array(np.linspace(0.01,1.0, testing_number), dtype='float32').reshape(testing_number, 1)

y_train = []
    
for idx in range(len(x_train)):
    y_train.append(call(x_train[idx], sigma))
y_train = np.array(y_train)

sk_kernel = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000.0)) #+ WhiteKernel(noise_level = 1e-12) #100000.0
gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel, n_restarts_optimizer=20)
gp.fit(x_train,y_train)
y_pred, sigma_hat = gp.predict(x_test, return_std=True)
    
l = gp.kernel_.length_scale
rbf= gaussian_process.kernels.RBF(length_scale=l)
 
Kernel= rbf(x_train, x_train)
K_y = Kernel + np.eye(training_number) * 1e-8
L = sp.linalg.cho_factor(K_y)
alpha_p = sp.linalg.cho_solve(np.transpose(L), y_train)
    
k_s = rbf(x_test, x_train)

k_s_prime = np.zeros([len(x_test), len(x_train)])
for i in range(len(x_test)):
    for j in range(len(x_train)):
        k_s_prime[i,j]=(1.0/l**2)*(x_train[j]-x_test[i])*k_s[i,j]
        
f_prime = np.dot(k_s_prime, alpha_p)/(ub-lb)

/Users/matthewdixon/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [35]:
delta = lambda x,y: bsformula(1, lb+(ub-lb)*x, KC, r, T, y, 0)[1]
delta(x_test, sigma) - f_prime

array([[ 4.53842113e-04],
       [ 3.72648539e-05],
       [-1.07730029e-04],
       [-9.17493393e-05],
       [-3.51949053e-05],
       [ 2.35446794e-05],
       [ 3.13013123e-05],
       [ 3.04747591e-05],
       [ 6.00280676e-06],
       [ 4.32545332e-06],
       [-6.05399460e-06],
       [-4.52778807e-06],
       [ 1.92118052e-05],
       [ 3.09640763e-05],
       [ 1.29672728e-05],
       [ 3.59283140e-06],
       [ 1.71721733e-05],
       [-5.26015015e-07],
       [-1.32361376e-05],
       [-1.74162712e-05],
       [-7.89467030e-06],
       [ 1.03357690e-05],
       [ 1.17970208e-05],
       [ 9.89195060e-06],
       [-2.08707341e-05],
       [-1.31494895e-05],
       [-3.21162721e-05],
       [-4.01014684e-05],
       [-6.45356722e-05],
       [-6.77282479e-05],
       [-7.16718331e-05],
       [-7.27188841e-05],
       [-7.60354533e-05],
       [-9.59807911e-05],
       [-1.11625311e-04],
       [-1.46763204e-04],
       [-1.63880787e-04],
       [-2.35246016e-04],
       [-2.0

## vega

In [36]:
x_train = np.array(np.linspace(0.01,1.0, training_number), dtype='float32').reshape(training_number, 1)
x_test = np.array(np.linspace(0.01,1.0, testing_number), dtype='float32').reshape(testing_number, 1)

y_train = []
    
for idx in range(len(x_train)):
    y_train.append(call((S0-lb)/(ub-lb), x_train[idx]))
y_train = np.array(y_train)

sk_kernel = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000.0)) #+ WhiteKernel(noise_level = 1e-12) #100000.0
gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel, n_restarts_optimizer=20)
gp.fit(x_train,y_train)
y_pred, sigma_hat = gp.predict(x_test, return_std=True)
    
l = gp.kernel_.length_scale
rbf= gaussian_process.kernels.RBF(length_scale=l)
 
Kernel= rbf(x_train, x_train)
K_y = Kernel + np.eye(training_number) * 1e-8
L = sp.linalg.cho_factor(K_y)
alpha_p = sp.linalg.cho_solve(np.transpose(L), y_train)
    
k_s = rbf(x_test, x_train)

k_s_prime = np.zeros([len(x_test), len(x_train)])
for i in range(len(x_test)):
    for j in range(len(x_train)):
        k_s_prime[i,j]=(1.0/l**2)*(x_train[j]-x_test[i])*k_s[i,j]
        
f_prime = np.dot(k_s_prime, alpha_p)

/Users/matthewdixon/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([6.85542814]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 84, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/Users/matthewdixon/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-4.89954874]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 61, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
/Users/matthewdixon/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [37]:
vega = lambda x,y: bsformula(1, lb+(ub-lb)*x, KC, r, T, y, 0)[2]
vega((S0-lb)/(ub-lb), x_test) - f_prime

array([[ 8.31393852e-04],
       [-2.11150622e-01],
       [ 5.15911460e-03],
       [ 2.88261559e-01],
       [ 2.61174789e-01],
       [-1.14445399e-02],
       [-1.91621194e-01],
       [-1.58682189e-01],
       [-4.13944979e-02],
       [ 6.84071987e-02],
       [ 9.36553344e-02],
       [ 5.79482420e-02],
       [-2.24028215e-03],
       [-2.15640144e-02],
       [-1.76980121e-02],
       [ 9.78432306e-03],
       [ 2.98610649e-02],
       [ 3.93100063e-02],
       [ 3.48635363e-02],
       [ 3.19368122e-02],
       [ 1.00333229e-02],
       [ 6.23524790e-04],
       [ 8.69684484e-03],
       [ 1.99237127e-02],
       [ 2.27766996e-02],
       [ 2.79133448e-02],
       [ 1.78532189e-02],
       [ 1.10985126e-02],
       [ 1.38075159e-03],
       [-3.65408531e-03],
       [ 2.84145874e-03],
       [ 1.24102153e-02],
       [ 9.73899774e-03],
       [ 1.26953022e-02],
       [ 1.24418713e-02],
       [ 7.70367074e-03],
       [-9.48703241e-03],
       [-7.70864578e-03],
       [-5.4